In [14]:
from data.gen_data import build_X, build_Y, build_XY
from data.gen_data_config import gen_data_config
from evaluation import score
import pandas as pd
import numpy as np
from util import plot_roi, categori_reverse
from model import build_model
from keras.optimizers import SGD, Adam, RMSprop, Adagrad
import keras
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM
from keras.layers.normalization import BatchNormalization

In [12]:
def conbine_LSTM_Dense(input_shape, output_shape):
    model = keras.models.Sequential()
    model.add(LSTM(256, return_sequences=True, input_shape=input_shape[1:]))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(128))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dense(output_shape[1]))
    model.add(Activation('softmax'))
    
    return model

In [3]:
batch_size = 128
epochs = 300

In [5]:
tr_s, tr_e = '2001/1/1', '2018/12/31'
te_s, te_e = '2019/1/1', '2019/12/31'
feature_days = 3

In [7]:
tr_x, tr_y, tr_result = build_XY(tr_s, tr_e, 
                                     gen_data_config['selected_features'], 
                                     feature_days, 2)
te_x, te_y, te_result = build_XY(te_s, te_e, 
                                 gen_data_config['selected_features'], 
                                 feature_days, 2)

In [10]:
result_roi = {
}

In [8]:
# convert class vectors to binary class matrices
one_tr_y = keras.utils.to_categorical(tr_y, 3)
one_te_y = keras.utils.to_categorical(te_y, 3)

In [9]:
re_tr_x = np.reshape(tr_x, (-1, feature_days, len(gen_data_config['selected_features'])))
re_te_x = np.reshape(te_x, (-1, feature_days, len(gen_data_config['selected_features'])))

In [17]:
model_combine = conbine_LSTM_Dense(re_tr_x.shape, one_tr_y.shape)
''' Compile model with specified loss and optimizer '''
model_combine.compile(loss= 'categorical_crossentropy',
                   optimizer='Adam',
                   metrics=['accuracy'])


'''Fit models and use validation_split=0.1 '''
history = model_combine.fit(re_tr_x, one_tr_y,
                              batch_size=batch_size,
                              epochs=epochs,
                              verbose=0,
                              shuffle=True,
                              validation_split=0.1,)

pred_y = model_combine.predict(re_te_x)
pred_z = categori_reverse(pred_y)
result_roi['combine'] = score(pred_z, te_result)

#2action_day: 102, #3consecutive_loss_max: -203, #4consecutive_loss_days_max: 9, #5loss_days: 68, roi: -188, roi_max: 144, roi_min: -271
